1) Spot-futures parity
    1.1) Получить данные котировок (исторические/лайв)
        1.1.1) Измерить доходность и риск активов
    1.2) Получить данные фьючерсов (исторические/лайв)
    1.3) Получить данные T-bills
    1.4) Рассчитать профит с учетом срока до экспирации и комиссий

2) Отработка спрэдов на криптобирже

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import scipy

from QuikPy import QuikPy
from techfunc import *
from stat_func import *

In [2]:
qp_provider = QuikPy()

In [3]:
govrate = 0.19
borrowlong = govrate+0.09
borrowshort = govrate+0.14

stockclass = 'TQBR'
futuresclass = 'SPBFUT'

Получение всех данных

In [6]:
minint = 5

moexf5 = qp_provider.get_candles_from_data_source(futuresclass, 'IMOEXF', minint)
brx45 = qp_provider.get_candles_from_data_source(futuresclass, 'BRX4', minint)
sber5 = qp_provider.get_candles_from_data_source(stockclass, 'SBER', minint)
gazp5 = qp_provider.get_candles_from_data_source(stockclass, 'GAZP', minint)
si5 = qp_provider.get_candles_from_data_source(futuresclass, 'SiZ4', minint)

moexf5 = collectdatequik(moexf5)[::-1].reset_index(drop=True)
brx45 = collectdatequik(brx45)[::-1].reset_index(drop=True)
sber5 = collectdatequik(sber5)[::-1].reset_index(drop=True)
gazp5 = collectdatequik(gazp5)[::-1].reset_index(drop=True)
si5 = collectdatequik(si5)[::-1].reset_index(drop=True)

dfclose5 = pd.DataFrame()
dfclose5['imoexf'] = moexf5[:672].close #/10
dfclose5['brx4'] = brx45[:672].close #*3.5
dfclose5['sber'] = sber5[:672].close
dfclose5['gazp'] = gazp5[:672].close #*2
dfclose5['si'] = si5[:672].close #/300

In [7]:
minint = 240

moexf4 = qp_provider.get_candles_from_data_source(futuresclass, 'IMOEXF', minint)
brx44 = qp_provider.get_candles_from_data_source(futuresclass, 'BRX4', minint)
sber4 = qp_provider.get_candles_from_data_source(stockclass, 'SBER', minint)
gazp4 = qp_provider.get_candles_from_data_source(stockclass, 'GAZP', minint)
si4 = qp_provider.get_candles_from_data_source(futuresclass, 'SiZ4', minint)

moexf4 = collectdatequik(moexf4)[::-1].reset_index(drop=True)
brx44 = collectdatequik(brx44)[::-1].reset_index(drop=True)
sber4 = collectdatequik(sber4)[::-1].reset_index(drop=True)
gazp4 = collectdatequik(gazp4)[::-1].reset_index(drop=True)
si4 = collectdatequik(si4)[::-1].reset_index(drop=True)

dfclose4 = pd.DataFrame()
dfclose4['imoexf'] = moexf4[:100].close #/10
dfclose4['brx4'] = brx44[:100].close #*3.5
dfclose4['sber'] = sber4[:100].close
dfclose4['gazp'] = gazp4[:100].close #*2
dfclose4['si'] = si4[:100].close #/300

Графики

In [ ]:
sns.set_style('whitegrid')
si4yield = getvar(si4)['frameyield']
sns.histplot(si4yield)

In [22]:
si4yield = getvar(si4[:100])['frameyield']
si4yield.describe()

d:\pyquik\techfunc.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['yield'] = (df['open'] - df['close'])/df['open']


count    100.000000
mean      -0.000432
std        0.003111
min       -0.008069
25%       -0.002209
50%       -0.000543
75%        0.001200
max        0.009218
Name: yield, dtype: float64

In [ ]:
moexf4yield = getyield(moexf4[:100])['yield']
moexf4yield.describe()

In [ ]:
sns.set_style('whitegrid')
sns.lineplot(dfclose4)

Корреляции и регрессии

In [ ]:
dfclose5.describe()

In [ ]:
dfclose4.std(axis=0)

In [ ]:
dfclose5.corr()

In [ ]:
dfclose4.corr()

Futures pricing

In [6]:
assetprice = qp_provider.get_quote_level2(stockclass, 'CHMF')['data']['bid'][-1]['price']
futureprice = qp_provider.get_quote_level2(futuresclass, 'CHZ4')['data']['bid'][-1]['price']

calcspread(float(assetprice), borrowlong, '2024-12-19', dividend=3400, futval=float(futureprice), lotsize=100)

(125600.95999999999, -1231.9599999999919)

In [7]:
type((pd.to_datetime('2024-12-19')-pd.to_datetime('today').normalize()).days)

int

In [24]:
sberfut

'27351'